In [6]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup
import time

In [ ]:
options = Options()
options.add_argument("--headless")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [10]:
service = Service()  
driver = webdriver.Edge(service=service, options=options)

In [11]:
url = 'https://www.cricbuzz.com/live-cricket-scorecard/35612/mi-vs-rcb-1st-match-indian-premier-league-2021'
driver.get(url)

In [ ]:
match_status = "completed"  # hardcoded for this match, else parse dynamically
match_datetime = soup.find('div', class_='cb-mtch-info-itm').text.strip()

In [12]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
match_data = {}
date_elem = soup.find('span', {'class': 'schedule-date ng-isolate-scope'})
match_data['match_date'] = date_elem.text.strip() if date_elem else None

In [ ]:
info_labels = soup.select('.cb-col.cb-col-27')
info_values = soup.select('.cb-col.cb-col-73')

for label, value in zip(info_labels, info_values):
    label_text = label.text.strip().lower()
    value_text = value.text.strip()
    if 'venue' in label_text:
        match_data['venue'] = value_text
    if 'umpires' in label_text:
        match_data['umpires'] = value_text
    if 'third umpire' in label_text:
        match_data['third_umpire'] = value_text
    if 'match referee' in label_text:
        match_data['match_referee'] = value_text


In [35]:
teams = soup.select('.cb-col.cb-col-100.cb-minfo-tm-nm')
if len(teams) >= 2:
    match_data['team1_name'] = teams[0].text.strip()
    match_data['team1_players'] = teams[1].text.strip()

In [37]:
squad_sections = soup.select('.cb-col.cb-col-100.cb-minfo-tm-nm ~ div')

team1_players = {"captain": None, "players": [], "bench": [], "support_staff": None}
team2_players = {"captain": None, "players": [], "bench": [], "support_staff": None}